# **Demo - Interfaccia I2C**

Il notebook mostra come usare $\texttt{Analog Discovery 2}$ per instaurare una connessione seriale di tipo $\texttt{I2C}$.

In questo caso lo script comunica con il sensore $\texttt{SHT3x}$ della Sensirion e utilizza i seguenti registri:

* `0x27` configurazione del sensore {\em high-repetibilty}
* `0xE0` Registro FETCH, richiede una lettura del sensore 

**Interpretazione della lettura**. Il sensore risponde con 6 bytes, i primi 2 bytes codificano la temperatura, nel datasheet è riportato il significato degli altri 4. I due bytes letti vengono combinati in un `uint16` che chiameremo `utemp`. La conversione finale richiede

        temp = -45 + 175 * utemp / (2**16-1)

**Lettura dell'umidità'**. può essere implementata in modo analogo, cercando il registro giusto e la conversione corretta

**Qualche accortezza**. Ricordiamo che il sensore fornisce i necessari *pull-up* per il corretto funzionamento dell'interfaccia I2C, quindi è fondamentale che il sensore sia alimentato altrimenti l'avvio del bus I2C darà un errore.

In [ ]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import time

# -[SHT3x - Qualche comando]---------------------------------------------------
HI10MPS = [0x27, 0x37]
FETCH = [0xE0, 0x00]
SAD = 0x44

# -[Configurazione AD2]--------------------------------------------------------
#   1. Connessiene con AD2 e selezione configurazione
ad2 = tdwf.AD2()
#   2. Configurazione alimentazione
ad2.vdd = 3.3
ad2.power(True)
#   3. Impostazione bus I2C
i2c = tdwf.I2Cbus(ad2.hdwf)  # default 100kHz, SCL = D00, SDA = D01
devs = i2c.scan()  # verifica cosa è connesso...
for dev in devs:
    print(f"Device: 0x{dev:02x}")
#   4. Apre la comunicazione I2C come Master
sht = tdwf.I2Cdevice(ad2.hdwf,SAD)

# -[Configurazione sensore]------------------------------------------------------
sht.write(HI10MPS)  # Imposta la configurazione 0x37 (a cosa corrisponde ?)

# -[Funzioni di gestione eventi]-----------------------------------------------

def on_close(event):
    global flag_run
    flag_run = False

def on_key(event):
    global flag_run
    if event.key == 'escape':  # termina programma
        flag_run = False

# -[Ciclo di misura]-----------------------------------------------------------
#   1. Creazione figura e link agli eventi
fig, ax = plt.subplots(figsize=(12,6))
fig.canvas.mpl_connect("close_event", on_close)
fig.canvas.mpl_connect("key_press_event", on_key)

#   2. Inizializzazione variabili
mytime = []
TTv = []
flag_first = True
flag_run = True

start_time = time.time()

#   3. Ciclo di misura
while flag_run:
    time.sleep(0.1)
    sht.writeread(FETCH,6)

    # Combinazione dei bytes letti
    # shift del byte HIGH di 8 posizioni + aggiunta del byte LOW
    TT = (sht.vals[0] << 8) + sht.vals[1]
    TT = -45 + 175*TT/(2**16-1) # Conversione temperatura
    TTv.append(TT)
    print(f"T = {TT:.2f}C")

    mytime.append(time.time()-start_time)
    if flag_first:
        flag_first = False
        hp1, = plt.plot(mytime,TTv, "o", color="tab:orange")
        plt.grid(True)
        plt.ylabel("Temperature [C]")
        plt.xlabel("Time [s]")
        plt.title("Press ESC to exit")
        plt.show(block=False)    
        plt.tight_layout()
    else:
        hp1.set_ydata(TTv)
        hp1.set_xdata(mytime)
        ax.relim()           
        ax.autoscale_view() 
        fig.canvas.draw()
        fig.canvas.flush_events()
# ---------------------------------------
plt.close(fig)
ad2.close()

Dispositivo #1 [SN:210321B19D0C, hdwf=1] connesso!
Configurazione #1
Bus I2C pronto...
Device: 0x44
T = 22.44C
T = 22.43C
T = 22.45C
T = 22.44C
T = 22.43C
T = 22.43C
T = 22.44C
T = 22.44C
T = 22.44C
T = 22.43C
T = 22.44C
T = 22.44C
T = 22.44C
T = 22.45C
T = 22.43C
T = 22.44C
T = 22.43C
T = 22.45C


KeyboardInterrupt: 